# Understanding LangChain Agents and Chains



### **What is LangChain?**
> LangChain is a framework for developing applications powered by language models.

 LangChain makes the complicated parts of working & building with AI models easier. It helps do this in two ways:

1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs
2. **Agency** - Allow your LLMs to interact with it's environment via decision making. Use LLMs to help decide which action to take next

### **Why LangChain?**
1. **Components** - LangChain makes it easy to swap out abstractions and components necessary to work with language models.

2. **Customized Chains** - LangChain provides out of the box support for using and customizing 'chains' - a series of actions strung together.

3. **Speed 🚢** - This team ships insanely fast. You'll be up to date with the latest LLM features.

4. **Community 👥** - Wonderful discord and community support, meet ups, hackathons, etc.



## Chains

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

## Agents

Agents can be thought of as the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. For a full list of built-in agents see agent types. You can also easily b


## Chains v/s Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.




In [1]:
# Installing necessary libraries
!pip install langchain-openai langchain langchain-core langchain-community --quiet
!pip install httpx langchain-experimental --quiet
!pip install openai google-search-results tiktoken wikipedia pyowm --quiet

# Chains with LangChain

In [2]:
api_key = "xxxxxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://xxxxxxxxx.openai.azure.com/"
model_name = "gpt-35-turbo"

In [3]:
import os
os.environ["OPENAI_API_VERSION"] = api_version
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_endpoint
os.environ["AZURE_OPENAI_API_KEY"] = api_key
os.environ["OPENWEATHERMAP_API_KEY"] = "29af1cea50a401d8e624eea4660b3f59"
os.environ["AZURE_OPENAI_API_VERSION"] = api_version

## Implementing a sample language translation chain

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.




In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import AzureChatOpenAI

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

#2. Initialize the LLM as a model
model = AzureChatOpenAI(azure_deployment=model_name,)

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


chain.invoke({"language": "italian", "text": "hi"})

'ciao'

In [5]:
chain.invoke({"language":"spanish","text":"THank You"})

'Gracias'

## Implementing a database Integration Chain

In [13]:
from langchain_openai import AzureChatOpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = AzureChatOpenAI(azure_deployment=model_name,)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

- Approach 1: input + prompt emplate (DB Schema) --> LLM ---> (outputs query)---> tool (DB tool to execute query) ---> LLM ---> Final Response

- Approach 2: input + user cred --> F0 ---> Fetch the DB schema based on accessible tables ---> Prompt template + DB schema ----> LLM ---> (outputs query)---> tool (DB tool to execute query) ---> LLM ---> Final Response

In [11]:
response = db_chain.run("What are tables in availabe in the database?")
print(response)

The tables available in the database are:
1. albums
2. sqlite_sequence
3. artists
4. customers
5. employees
6. genres
7. invoices
8. invoice_items
9. media_types
10. playlists
11. playlist_track
12. tracks
13. sqlite_stat1


In [9]:
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


'There are 8 employees.'

In [15]:
db_chain.run("How many unique employees we have and what are unique playlists?")



> Entering new SQLDatabaseChain chain...
How many unique employees we have and what are unique playlists?
SQLQuery:SELECT COUNT(DISTINCT EmployeeId) AS UniqueEmployees, COUNT(DISTINCT PlaylistId) AS UniquePlaylists
FROM employees, playlists
SQLResult: [(8, 18)]
Answer:We have 8 unique employees and 18 unique playlists.
> Finished chain.


'We have 8 unique employees and 18 unique playlists.'

# LangChain AI Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.



## Creating a sample LangChain Agent

# Zero-Shot ReAct Agent

In [16]:
# Importing necessary modules for the agent
from langchain.agents import load_tools      # Imports the load_tools function for loading various tools in the langchain package
from langchain_openai import AzureChatOpenAI # Imports the ChatOpenAI class to interact with OpenAI's models


In [17]:
# Initializing a Language Model (LLM) instance
model = AzureChatOpenAI(azure_deployment=model_name,)

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)


In [ ]:
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage


response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

### Implementing Agent with LangChain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)
prompt

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "hi!"})

In [ ]:
agent_executor.invoke({"input": "What's the weather in SF?"})

## Implementing a Multitool Agent

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor


prompt = ChatPromptTemplate.from_messages(
    [   ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"), ])

search = TavilySearchResults(max_results=2)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# Create Image Gentation model Object

tools = [search, wikipedia]

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What's the current weather in SF?"})

In [ ]:
agent_executor.invoke({"input": "What is capital of Indonesia?"})

## Thank You